# GPU 사용가능한지 확인

In [3]:
import torch

# GPU 사용 가능 -> True, GPU 사용 불가 -> False
print(torch.cuda.is_available())

False


# YOlov8n모델 예측 후 객체의 정보 얻기

In [4]:
from ultralytics import YOLO
import torch

# 모델 불러오기 없으면 다운받기
model = YOLO('yolov8n.pt')

# 예측할 이미지 or 동영상
path = './sample.jpg'

# 예측 결과
results = model.predict(path, save=True)

# 객체의 데이터
for result in results:
    # 객체의 바운딩 박스 결과
    boxes = result.boxes

# 클래스 ID
object_type = []
for _ in boxes.cls:
    object_type.append(int(_))

# 객체의 좌표
yolo_point = []
for _ in boxes.xywh:
    center_x = int(_[0])
    center_y = int(_[1])
    width = int(_[2])
    hight = int(_[3])

    yolo_point.append((center_x, center_y, width, hight))


image 1/1 c:\study\blackbox\sample.jpg: 640x640 3 persons, 1 car, 157.5ms
Speed: 5.0ms preprocess, 157.5ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict3


In [5]:
result_path = './' + result.save_dir.replace('\\', '/') + '/'
result_path

'./runs/detect/predict3/'

In [6]:
# Opencv 로 시각화 할때 필요한 예측 결과 이미지 or 예측 동영상
import os

img_file = os.listdir(result_path)
show_result = result_path + img_file[0]

In [7]:
# coco 데이터 구조 파악
result.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [8]:
object_type

[0, 0, 0, 2]

In [9]:
len(object_type), len(yolo_point)

(4, 4)

In [10]:
object_type

[0, 0, 0, 2]

In [11]:
yolo_point

[(295, 372, 23, 99),
 (411, 384, 34, 170),
 (440, 366, 38, 164),
 (528, 329, 39, 43)]

### 이미지내 불필요한 클래스 제거

In [12]:
# 불필요한 클래스id 인덱스 받기
drop_index = []

# 0 : 사람, 1 : 자전거, 2 : 자동차
# 3 : 오토바이, 5 : 버스, 7 : 트럭
for _ in range(0, len(object_type)):
    if object_type[_] == 0 or object_type[_] == 1 or object_type[_] == 2 or object_type[_] == 3 or object_type[_] == 5 or object_type[_] == 7:
        pass
    else:
        drop_index.append(_)

# 불필요한 클래스 인덱스 삭제
# 인덱스 번호를 안 뒤집을 경우 앞에서부터 삭제하므로
# 뒤로갈수록 해당 인덱스 존재 안해서.
drop_index.reverse()
for _ in drop_index:
    object_type.pop(_)
    yolo_point.pop(_)

### 클래스 id 정수형 -> 한글로 변환

In [13]:
# 클래스id -> 한글로 변환
for _ in range(0, len(object_type)):
    if object_type[_] == 0:
        object_type[_] = '사람'

    elif object_type[_] == 1:
        object_type[_] = '자전거'

    elif object_type[_] == 2:
        object_type[_] = '자동차'

    elif object_type[_] == 3:
        object_type[_] = '오토바이'

    elif object_type[_] == 5:
        object_type[_] = '버스'

    elif object_type[_] == 7:
        object_type[_] = '트럭'       

object_type


['사람', '사람', '사람', '자동차']

### Yolo식 좌표는 center_x, center_y, w, h -> 바운딩박스 4개의 점 좌표로 변환

In [14]:
# 사각형의 4개의 점 좌표 구하기.
# x1 : 왼쪽 상단, x2 : 오른쪽 상단
# x3 : 왼쪽 하단, x4 : 오른쪽 하단

# 하나의 클래스 기준 4개의 점 좌표 저장.
point = []

for raw in yolo_point:
    x = raw[0]
    y = raw[1]
    w = raw[2]
    h = raw[3]
    x1 = (int(x - (w/2)), int(y + (h/2)))
    x2 = (int(x + (w/2)), int(y + (h/2)))
    x3 = (int(x - (w/2)), int(y - (h/2)))
    x4 = (int(x + (w/2)), int(y - (h/2)))

    point.append((x1, x2, x3, x4))

In [15]:
point

[((283, 421), (306, 421), (283, 322), (306, 322)),
 ((394, 469), (428, 469), (394, 299), (428, 299)),
 ((421, 448), (459, 448), (421, 284), (459, 284)),
 ((508, 350), (547, 350), (508, 307), (547, 307))]

In [16]:
point[0]

((283, 421), (306, 421), (283, 322), (306, 322))

### 알고리즘 구현

In [17]:
# opencv식 워닝존 좌표
warning_point = [(270, 310), (12, 445), (610, 445), (383, 310)]

# 변환 워닝존 좌표
warning_point = [(12, 310), (270, 445), (383, 445), (610, 310)]

In [18]:
# !pip install shapely

In [19]:
from shapely.geometry import Polygon, Point

def warning_zone(point, polygon):
    """
    point: [x, y] 좌표 값을 가진 리스트
    polygon: 다각형의 꼭지점 좌표를 가진 리스트
    """

    # 점의 좌표를 사용하여 점의 객체를 만듭니다.
    point_obj = Point(point)

    # 다각형의 꼭지점 좌표를 사용하여 다각형의 경계를 구합니다.
    polygon_path = Polygon(polygon)

    # 점의 객체가 다각형의 경계 안에 있는지 확인합니다.
    result = polygon_path.contains(point_obj)
    print(result)
    # print('워닝존 : ', polygon)
    return result




# 워닝존
rectangle = [(280, 290), (20, 470), (620, 470), (380, 290)]

# 데이터 클래스 하나 4개의 점
# square = point

result_case = []
# 좌표가 사다리꼴 안에 있는지 확인합니다.
for square, result_type in zip(point, object_type):
    for p in square:
        point_obj = Point(p)
        if warning_zone(point_obj, rectangle):
            # print(_, "포함됨")
            # 조건문
            if p[0] < int((rectangle[3][0] - rectangle[0][0]) / 2):
                result_txt = '왼쪽'
                print(f'{result_txt}에 {result_type}있습니다.')
                result_case.append([result_txt, result_type])
                break
            elif int((rectangle[3][0] - rectangle[0][0]) / 2) < p[0]:
                result_txt = '오른쪽'
                result_case.append([result_txt, result_type])
                print(f'{result_txt}에 {result_type}있습니다.')
                break
        else:
            pass
            # print("포함되지 않음")

result_case


True
오른쪽에 사람있습니다.
True
오른쪽에 사람있습니다.
True
오른쪽에 사람있습니다.
False
False
False
False


[['오른쪽', '사람'], ['오른쪽', '사람'], ['오른쪽', '사람']]

In [20]:
for _ in range(0, len(result_case)):
    a = result_case[_][0]
    ob = result_case[_][1]

이미지 테스트

In [23]:
import os
import cv2
import numpy as np
import sys

filePath  = show_result

img = cv2.imread(filePath)

w, h, c = img.shape

mask = np.zeros((w,h,c), dtype=np.uint8)

# pts1 = np.array([[270, 310], [12, 445], [610, 445], [383, 310]])
# pts1 = np.array([[12, 310], [270, 445], [383, 445], [610, 310]])
pts1 = np.array([[280, 290], [20, 470], [620, 470], [380, 290]])



desired_alpha = 90 
alpha = desired_alpha / 255.0 

# 다각형 그리기
polyline = cv2.polylines(mask, [pts1], isClosed=True, color=(0, 0, 255), thickness=5)

# 마스크 채우기
mask1 = cv2.fillPoly(polyline, [pts1], (255,0,0))
img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
mask1 = cv2.cvtColor(mask1, cv2.COLOR_BGR2BGRA)
result = cv2.addWeighted(img, 1, mask1, alpha, 0)

# 이미지 화면 출력
cv2.imshow('img', result)

# 이미지 저장
cv2.imwrite('./result.jpg', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 버전 - 1
워닝존에 객체 존재시 알고리즘 구현(워닝존 안에 객체가 있는지 -> 워닝존 밑변 기준 x 값 왼쪽에 있는지, 오른쪽에 있는지)

In [ ]:
from shapely.geometry import Polygon, Point

def warning_zone(point, polygon):
    """
    point: [x, y] 좌표 값을 가진 리스트
    polygon: 다각형의 꼭지점 좌표를 가진 리스트
    """

    # 점의 좌표를 사용하여 점의 객체를 만듭니다.
    point_obj = Point(point)

    # 다각형의 꼭지점 좌표를 사용하여 다각형의 경계를 구합니다.
    polygon_path = Polygon(polygon)

    # 점의 객체가 다각형의 경계 안에 있는지 확인합니다.
    result = polygon_path.contains(point_obj)
    print(result)
    # print('워닝존 : ', polygon)
    return result

# 워닝존
rectangle = [(280, 290), (20, 470), (620, 470), (380, 290)]

# 데이터 클래스 하나 4개의 점
# square = point

result_case = []
# 좌표가 사다리꼴 안에 있는지 확인합니다.
for square, result_type in zip(point, object_type):
    for p in square:
        point_obj = Point(p)
        if warning_zone(point_obj, rectangle):
            # print(_, "포함됨")
            # 조건문
            if p[0] < int((rectangle[3][0] - rectangle[0][0]) / 2):
                result_txt = '왼쪽'
                print(f'{result_txt}에 {result_type}있습니다.')
                result_case.append([result_txt, result_type])
                break
            elif int((rectangle[3][0] - rectangle[0][0]) / 2) < p[0]:
                result_txt = '오른쪽'
                result_case.append([result_txt, result_type])
                print(f'{result_txt}에 {result_type}있습니다.')
                break
        else:
            pass
            # print("포함되지 않음")

result_case